In [1]:
cd /Users/angli/Documents/GitHub/ToxicCommentsClassification

/Users/angli/Documents/GitHub/ToxicCommentsClassification


In [2]:
ls

All_Wordcloud.py*
CleanCommentGenerator.py
Comment2VecPlot.py
CommentsFeaturesGenerator.py
DataExplorationPrediction_Features.ipynb
DataExplorationPrediction_w2v.ipynb
README.md
ToxicCommentAnalysis.ipynb
Word2Vec_merge.py
Word2Vec_plot.py
cabin-sketch-v1.02/
report_update.docx*
usingW2V.py
w2v_train.py
word_cloud_pngs/
wordcloud.pyc*
x_tsne.csv
x_tsne.png


In [3]:
import sklearn
print (sklearn.__version__)

0.19.1


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import log_loss, f1_score, accuracy_score
from sklearn.neural_network import MLPClassifier

In [5]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import gridspec
import seaborn as sns
sns.set_style("dark")
sns.set(font_scale=1.3)

In [7]:
train = pd.read_csv("/Users/angli/Ang/OneDrive/Documents/Pitt_PhD/Class/2018Spring/ML/finalProject/data/train_features.csv")

In [9]:
w2v = pd.read_csv("/Users/angli/Ang/OneDrive/Documents/Pitt_PhD/Class/2018Spring/ML/finalProject/data/c2wdata.csv")

In [10]:
train = pd.merge(train, w2v, on='id')

In [11]:
train.shape

(159491, 224)

In [12]:
train.columns.values

array(['id', 'total_length', 'capitals', 'caps_vs_length',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'num_we', 'num_words', 'num_unique_words',
       'words_vs_unique', 'num_smilies', 'num_IP', 'num_URL',
       'num_positive', 'num_negtive', 'score', 'toxic', 'severe_toxic',
       'obscene', 'threat', 'insult', 'identity_hate', '0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
       '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70',
       '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81',
       '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92',
       '93

In [13]:
#make the label as binary classification
rowsums=train.iloc[:,18:24].sum(axis=1)
#sum(list(rowsums>0))
train['dirty']=(rowsums>0)
train['dirty'].sum()

16225

### prediction

In [14]:
y = train['dirty']
x = train[['total_length', 'capitals', 'caps_vs_length',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'num_we', 'num_words', 'num_unique_words',
       'words_vs_unique', 'num_smilies', 'num_IP', 'num_URL',
       'num_positive', 'num_negtive', 'score', '0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
       '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70',
       '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81',
       '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92',
       '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
       '103', '104', '105', '106', '107', '108', '109', '110', '111',
       '112', '113', '114', '115', '116', '117', '118', '119', '120',
       '121', '122', '123', '124', '125', '126', '127', '128', '129',
       '130', '131', '132', '133', '134', '135', '136', '137', '138',
       '139', '140', '141', '142', '143', '144', '145', '146', '147',
       '148', '149', '150', '151', '152', '153', '154', '155', '156',
       '157', '158', '159', '160', '161', '162', '163', '164', '165',
       '166', '167', '168', '169', '170', '171', '172', '173', '174',
       '175', '176', '177', '178', '179', '180', '181', '182', '183',
       '184', '185', '186', '187', '188', '189', '190', '191', '192',
       '193', '194', '195', '196', '197', '198', '199']]

In [32]:
#train, test 7/3
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=42, 
                                                  test_size=0.3, shuffle=True)

In [16]:
ytrain.sum()/len(ytrain)

0.10126922422364143

In [17]:
#logistic regression
model = LogisticRegression(penalty='l2')
model.fit(xtrain,ytrain)

prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.9331888250942737
Training F1 score: 0.5347139916411952
Testing Accuracy Rate: 0.9311987961879284
Testing F1 score: 0.5237268518518519


In [18]:
#naive bayes
model = BernoulliNB()
model.fit(xtrain,ytrain)

prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.9111901328341231
Training F1 score: 0.6052160063706947
Testing Accuracy Rate: 0.9103201805718107
Testing F1 score: 0.6057877813504823


In [28]:
#tree
model = tree.DecisionTreeClassifier(criterion='gini', min_samples_split=5, min_samples_leaf=3)
model.fit(xtrain,ytrain)

prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.986958429995611
Training F1 score: 0.9346675042627659
Testing Accuracy Rate: 0.9147508777796355
Testing F1 score: 0.5801338136901698


In [33]:
#bagging of decision tree -- entropy
base = tree.DecisionTreeClassifier(criterion='entropy')
model = BaggingClassifier(base_estimator=base, n_estimators=50, random_state=1, n_jobs=5)
model.fit(xtrain,ytrain)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=5, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [34]:
prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.9997312863323271
Training F1 score: 0.9986715082809317
Testing Accuracy Rate: 0.9485453937468651
Testing F1 score: 0.6937810945273631


In [35]:
#bagging of decision tree -- gini
base = tree.DecisionTreeClassifier(criterion='gini')
model = BaggingClassifier(base_estimator=base, n_estimators=50, random_state=1, n_jobs=5)
model.fit(xtrain,ytrain)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=5, oob_score=False,
         random_state=1, verbose=0, warm_start=False)

In [36]:
prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.9996417151097695
Training F1 score: 0.9982278929647351
Testing Accuracy Rate: 0.9467271359304464
Testing F1 score: 0.6810959589640935


In [37]:
#adaboost of decision tree
base = tree.DecisionTreeClassifier(criterion='gini')
model = AdaBoostClassifier(base_estimator=base, n_estimators=50)
model.fit(xtrain,ytrain)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [38]:
prd_test = model.predict(xtest)
#prd_test_label = prd_test.astype(int)
    
prd_train = model.predict(xtrain)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest,prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 1.0
Training F1 score: 1.0
Testing Accuracy Rate: 0.9096513960876108
Testing F1 score: 0.570747691391123


In [27]:
##Neural Network

In [19]:
#pre-process for nn, scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(xtrain)
X_train = scaler.transform(xtrain)
X_test = scaler.transform(xtest)

In [29]:
modelNN = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100,100,100,100), activation = "relu", solver='lbfgs', alpha=0.0001, 
                    #learning_rate = 'constant', learning_rate_init = 0.001, 
                    max_iter=500)

In [30]:
modelNN.fit(X_train, ytrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100),
       learning_rate='constant', learning_rate_init=0.001, max_iter=500,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [31]:
prd_test = modelNN.predict(X_test)
#prd_test_label = prd_test.astype(int)
    
prd_train = modelNN.predict(X_train)
#prd_train_label = prd_train.astype(int)
    
train_accuracy = accuracy_score(ytrain, prd_train)
train_f1 = f1_score(ytrain, prd_train)
test_accuracy = accuracy_score(ytest, prd_test)
test_f1 = f1_score(ytest,prd_test)
    
print('Training Accuracy Rate:', train_accuracy)
print('Training F1 score:', train_f1)
print('Testing Accuracy Rate:', test_accuracy)
print('Testing F1 score:', test_f1)

Training Accuracy Rate: 0.9998208575548848
Training F1 score: 0.9991155921110817
Testing Accuracy Rate: 0.9439683999331215
Testing F1 score: 0.7234086454142165
